In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import math
import statistics
import pickle
import multiprocessing
from multiprocessing import Pool, Manager

In [2]:
def get_pressure_weighted(x):
    dPref = (x.plev.values[0]-x.plev.values[-1])  #(p-ps)
    integral = []
    for i in range(len(x.plev)): #Integral of variable from P to Ps calculated as area between each pressure variable trapezoid then summed
        if i+1 < len(x.plev):
            area=((x.loc[dict(plev=x.plev.values[i])] + x.loc[dict(plev=x.plev.values[i+1])])/2)*(x.plev.values[i]-x.plev.values[i+1])
            integral.append(area)
    pw = (sum(integral))/dPref
    return(pw)


In [3]:
def seasoning(month):
    if (month == 12) | (month == 1) | (month == 2):
        return(1)
    if (month == 3) | (month == 4) | (month == 5):
        return(2)
    if (month == 6) | (month == 7) | (month == 8):
        return(3)
    if (month == 9) | (month == 10) | (month == 11):
        return(4)


In [4]:
def low_pass_weights(window, cutoff):
    order = ((window - 1) // 2 ) + 1
    nwts = 2 * order + 1
    w = np.zeros([nwts])
    n = nwts // 2
    w[n] = 2 * cutoff
    k = np.arange(1., n)
    sigma = np.sin(np.pi * k / n) * n / (np.pi * k)
    firstfactor = np.sin(2. * np.pi * cutoff * k) / (np.pi * k)
    w[n-1:0:-1] = firstfactor * sigma
    w[n+1:-1] = firstfactor * sigma
    return w[1:-1]

In [5]:
def getrange(numbers):
    return max(numbers) - min(numbers)


In [6]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [14]:
def jettracker(model,file,levels=[85000,70000],time=['1950', '2005'],infer = np.linspace(-75, 15, 501)):
    d = xr.open_mfdataset(file)
    print(model)
    if model == 'NOAA':
        d = d.rename({'uwnd':'ua'})
        d = d.rename({'level':'plev'})
        levels=[850,700]
    elif model == 'ERA5':
        d = d.rename({'latitude':'lat'})
        d = d.rename({'longitude':'lon'})
        d = d.rename({'level':'plev'})
        levels=[850,700]
        time=['1980','2018']
    d = xr.concat([d.sel(lon = slice(0,30)),d.sel(lon = slice(320,360))],dim='lon')
    d.coords['lon'] = (d.coords['lon'] + 180) % 360 - 180
    if model == 'ERA5':
        d = d.sel(lat = slice(-15,-75))
        d = d.sel(time = slice('1980','2018'))
    else:
        d = d.sel(lat = slice(-75,-15))
    wgts = low_pass_weights(41, 1/10)
    weight = xr.DataArray(list(wgts), dims=['window'])
    _, index = np.unique(d['time'], return_index=True)
    d = d.isel(time=index)
    d = d.sel(plev=levels,method='nearest')
    d = d.sel(time=slice(time[0], time[-1]))
    d = get_pressure_weighted(d.ua)
    d = d.rolling(time=41, center=True).construct('window').dot(weight)
    d = d[20:-21]
    d = d.mean(dim='lon',skipna=True)
    d = d.sortby(d.lat)
    #d = d.load() #this is faster than loading later but ERA5 reaches a RAM limit - Memory leak from open_mfdataset also a recurring problem
    out = []
    size = len(d.lat)-1
    for y in pd.date_range(str(int(time[0])+1),time[-1], freq='A'): #1980-2004
        a = d.sel(time = str(y.year))
        a = a.load()
        b = a.where(a==a.max(dim='lat')) #find max zonal mean/set rest to Nan
        for i in range(len(a.values)): #each day
            fwhm=[]
            for j in range(len(a.values[i])): #each lat
                if math.isnan(b.values[i][j])==False: #if a maximum is found
                    if j==0 or j==size:
                        lat = a.lat.values[j]
                        ua = b.values[i][j]
                        break
                    else:
                        p = np.poly1d(np.polyfit(b.lat.values[j-1:j+2],a.values[i][j-1:j+2],2)) #fit quadratic to interpolate
                        lat = pd.DataFrame(p(infer),index=infer).idxmax().values[0]
                        ua = pd.DataFrame(p(infer),index=infer).max().values[0]
                        break
            for k in range(len(b.lat.values)): #check against each lat to find width
                if a.values[i][k] >= ua/2:
                    fwhm.append(k) #full width half mean
            if len(fwhm) >= 2: #some days no latitudes other than maximum meet fwhm criteria
                if fwhm[0]==0: #if bottom is maximum latitude else interpolate with linear model
                    bottom = b.lat.values[fwhm[0]]
                else:
                    wb = np.poly1d(np.polyfit(b.lat.values[fwhm[0]-1:fwhm[0]+2],a.values[i][fwhm[0]-1:fwhm[0]+2],1))
                    inf = infer
                    bottom = inf[find_nearest(wb(inf),ua/2)]
                if fwhm[-1]==size: #if top is minimum latitude else interpolate with linear model
                    top = b.lat.values[fwhm[-1]]
                else:
                    wt = np.poly1d(np.polyfit(b.lat.values[fwhm[-1]-1:fwhm[-1]+2],a.values[i][fwhm[-1]-1:fwhm[-1]+2],1))
                    inf = infer
                    top = inf[find_nearest(wt(inf),ua/2)]
                width = top - bottom
            else:
                width = np.nan
            out.append([a[i].time.values,lat,ua,width,top,bottom]) #append to list
            a.close()
    df = pd.DataFrame(np.array(out),columns=['time', 'lat', 'ua','width','top','bottom'])
    return df



In [8]:
def get_files():
    models = glob.glob("/terra/data/cmip5/global/historical/*")
    avail={}
    for model in models:
        ua = glob.glob(str(model)+"/r1i1p1/day/native/ua_*")
        try:
            test = ua[0]
            avail[model.split('/')[-1]] = ua
        except:
             pass
    return avail

In [9]:
files = get_files()

In [10]:
files['NOAA'] =  glob.glob("/terra/data/reanalysis/global/reanalysis/NOAA/20thC/r1/day/native/ua_*")

In [11]:
files['ERA5'] = glob.glob("/terra/data/reanalysis/global/reanalysis/ECMWF/ERA5/day/native/ua_*")

In [15]:
dic = {}
for model in dic:
    dic[model] = jettracker(model,files[model])

ERA5


In [16]:
pr_files = {}
for index in dic:
    if index == 'NOAA':
        pr_files[index] = glob.glob("/terra/data/reanalysis/global/reanalysis/NOAA/20thC/r1/day/native/pr*")
    elif index == 'ERA5':
        pr_files[index] = glob.glob("/terra/data/reanalysis/global/reanalysis/ECMWF/ERA5/day/native/pr*")
    else:
        pr_files[index] = glob.glob("/terra/data/cmip5/global/historical/"+str(index)+"/r1i1p1/day/native/pr*")

In [19]:
for model in dic:
    print(model)
    x = xr.open_mfdataset(pr_files[model])
    _, index = np.unique(x['time'], return_index=True)
    x = x.isel(time=index)
    if model == 'ERA5':
        x = x.sel(time = slice('1981','2017'))
        x = x.sel(latitude=-34,method='nearest')
        x = x.sel(longitude=18,method='nearest')        
    else:
        x = x.sel(time=slice('1951', '2004'))
        x = x.sel(lat=-34,method='nearest')
        x = x.sel(lon=18,method='nearest')
    if model == 'NOAA':
        x = x.prate.values
    else:
        x = x.pr.values
    dic[model]['pr'] = x

IPSL-CM5A-LR
MPI-ESM-MR
MRI-ESM1
MIROC-ESM-CHEM
EC-EARTH
HadGEM2-CC
bcc-csm1-1-m
FGOALS-g2
ACCESS1-0
MIROC-ESM
CSIRO-Mk3-6-0
GFDL-CM3
CanESM2
IPSL-CM5B-LR
MPI-ESM-LR
CMCC-CM
GFDL-ESM2G
MIROC5
MPI-ESM-P


/share/apps/anaconda3/envs/dask/lib/python3.8/site-packages/xarray/coding/times.py:113: SerializationWarning: Ambiguous reference date string: 850-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


CMCC-CMS
CNRM-CM5
IPSL-CM5A-MR
inmcm4
ACCESS1-3
MRI-CGCM3
bcc-csm1-1
CMCC-CESM
HadGEM2-AO
BNU-ESM
NorESM1-M
HadCM3
GFDL-ESM2M
MIROC4h
NOAA
ERA5


In [24]:
arraytype = type(dic['bcc-csm1-1-m'].time[0])
for index in dic:
    years=[]
    months=[]
    print(index)
    for i in range(len(dic[index])):
        if isinstance(dic[index].time[i], arraytype):
            years.append(dic[index].time[i].ravel()[0].year)
            months.append(dic[index].time[i].ravel()[0].month)
        else:
            years.append(dic[index].time[i].year)
            months.append(dic[index].time[i].month)
    dic[index]['years'] = years
    dic[index]['months'] = months
for index in dic:
    dic[index]['seasons'] = [seasoning(i) for i in dic[index].months.values]

IPSL-CM5A-LR
MPI-ESM-MR
MRI-ESM1
MIROC-ESM-CHEM
EC-EARTH
HadGEM2-CC
bcc-csm1-1-m
FGOALS-g2
ACCESS1-0
MIROC-ESM
CSIRO-Mk3-6-0
GFDL-CM3
CanESM2
IPSL-CM5B-LR
MPI-ESM-LR
CMCC-CM
GFDL-ESM2G
MIROC5
MPI-ESM-P
CMCC-CMS
CNRM-CM5
IPSL-CM5A-MR
inmcm4
ACCESS1-3
MRI-CGCM3
bcc-csm1-1
CMCC-CESM
HadGEM2-AO
BNU-ESM
NorESM1-M
HadCM3
GFDL-ESM2M
MIROC4h
NOAA
ERA5


In [25]:
pickle.dump(dic, open( "../JET_OUT/jettrack_1D.p", "wb" ))